# Fashion Concierge manual testing (all phases)

This notebook walks through manual checks for the Fashion Concierge backend from Phase One through Phase Five. Each section explains what is being exercised and uses local mocks so the workflow can be run entirely offline.


## Introduction and setup

We import the core data models, tools, and agents needed for the manual checks. The Python path is adjusted so modules such as `adk_app`, `agents`, `tools`, `models`, and `logic` can be imported when running the notebook from the repository root.


In [ ]:
# Standard library imports
from __future__ import annotations
import logging
import sys
from dataclasses import asdict
from datetime import date, datetime, timedelta
from pathlib import Path
from typing import Dict, List

# Ensure the repository root is on sys.path so local packages can be imported.
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

logging.basicConfig(level=logging.INFO, format='%(levelname)s %(name)s: %(message)s')

# Core taxonomy and models
from models.taxonomy import CATEGORIES, MOODS, validate_category, validate_subcategory
from models.wardrobe_item import WardrobeItem

# Wardrobe storage
from tools.wardrobe_store import SQLiteWardrobeStore
from tools.wardrobe_tools import WardrobeTools

# URL ingestion helpers
from tools.product_parser import parse_product_html
from models.ingestion_mapping import map_raw_metadata_to_wardrobe_item

# Mood and color theory helpers
from models.mood_styles import get_mood_style
from models.color_theory import complementary, monochrome, analogous_triplet, choose_harmonious_colors

# Calendar and weather providers (mocked for offline use)
from tools.calendar_provider import CalendarEvent, MockCalendarProvider
from tools.weather_provider import MockWeatherProvider, WeatherProfile

# Context synthesizer and outfit logic
from logic.context_synthesizer import synthesize_context
from logic.contextual_filtering import filter_by_formality, filter_by_movement, filter_by_mood, filter_by_weather
from logic.outfit_builder import (
    CandidateSelectionResult,
    CollageSpecResult,
    HarmonyApplicationResult,
    OutfitBuildResult,
    apply_color_harmony,
    build_outfit,
    generate_collage_spec,
    select_candidates_for_mood,
)
from logic.outfit_scoring import score_outfit

# Agents and orchestrator
from adk_app.config import ADKConfig
from agents.calendar_agent import CalendarAgent
from agents.weather_agent import WeatherAgent
from agents.outfit_stylist_agent import OutfitStylistAgent
from agents.orchestrator import OrchestratorAgent

# Shared demo constants
USER_ID = 'demo_user'
DB_PATH = Path('tests/manual_demo/wardrobe.db')
DB_PATH.parent.mkdir(parents=True, exist_ok=True)


## Wardrobe taxonomy and storage (Phase One)

Validate taxonomy helpers, then exercise the local wardrobe store by inserting, updating, listing, and deleting items for a demo user. These operations show how deterministic storage works without any external dependencies.


In [ ]:
        # Explore canonical categories and validate values
        print('Categories and subcategories:')
        for category, subs in CATEGORIES.items():
            print(f"- {category}: {', '.join(subs)}")

        valid_category = validate_category('Top')
        valid_subcategory = validate_subcategory(valid_category, 'tee')
        print(f"
Validated category/subcategory: {valid_category}/{valid_subcategory}")

        try:
            validate_category('made_up_category')
        except ValueError as exc:
            print(f'Invalid category check correctly failed: {exc}')


In [ ]:
        # Instantiate a fresh SQLite wardrobe store using a temporary path
        store = SQLiteWardrobeStore(DB_PATH)
        wardrobe_tools = WardrobeTools(store)

        # Seed a few wardrobe items for the demo user
        seed_items: List[WardrobeItem] = [
            WardrobeItem(
                item_id='top_basic',
                user_id=USER_ID,
                image_url='http://example.com/top_basic.jpg',
                source_url='http://example.com/top_basic',
                category='top',
                sub_category='tee',
                colors=['blue'],
                style_tags=['casual'],
                season_tags=['all_year'],
            ),
            WardrobeItem(
                item_id='bottom_denim',
                user_id=USER_ID,
                image_url='http://example.com/bottom_denim.jpg',
                source_url='http://example.com/bottom_denim',
                category='bottom',
                sub_category='jeans',
                colors=['black'],
                style_tags=['casual'],
                season_tags=['all_year'],
            ),
            WardrobeItem(
                item_id='sneaker_white',
                user_id=USER_ID,
                image_url='http://example.com/sneaker_white.jpg',
                source_url='http://example.com/sneaker_white',
                category='shoes',
                sub_category='sneakers',
                colors=['white'],
                style_tags=['casual'],
                season_tags=['all_year'],
            ),
            WardrobeItem(
                item_id='outer_jacket',
                user_id=USER_ID,
                image_url='http://example.com/outer_jacket.jpg',
                source_url='http://example.com/outer_jacket',
                category='outerwear',
                sub_category='jacket',
                colors=['navy'],
                materials=['cotton'],
                style_tags=['street'],
                season_tags=['cold_weather'],
            ),
        ]

        for item in seed_items:
            store.create_item(item)

        print('Seeded items for demo user:')
        for item in store.list_items_for_user(USER_ID):
            print(f'- {item.item_id}: {item.category}/{item.sub_category} colors={item.colors}')

        # Update a record
        store.update_item(USER_ID, 'outer_jacket', {'colors': ['navy', 'gray'], 'user_notes': 'Add scarf when cold'})
        print('
After update:')
        for item in store.list_items_for_user(USER_ID):
            if item.item_id == 'outer_jacket':
                print(f'- {item.item_id} updated colors -> {item.colors}; notes={item.user_notes}')

        # Delete a record
        deleted = store.delete_item(USER_ID, 'bottom_denim')
        print(f'
Deleted bottom_denim? {deleted}')
        print('Current wardrobe:')
        for item in store.list_items_for_user(USER_ID):
            print(f'- {item.item_id}')


## URL ingestion (Phase Two)

Simulate ingestion with a static HTML fixture instead of live retailer pages. The fetch step is mocked; we parse the HTML, map the raw metadata to a `WardrobeItem`, and save it to the same local store.


In [ ]:
# Static HTML fixture representing a retailer page
        fixture_html = '''
<html>
<head>
  <meta property="og:title" content="Linen Blend Utility Jacket"/>
  <meta property="og:image" content="/images/utility-jacket.jpg"/>
  <meta name="description" content="Lightweight linen jacket in sand color"/>
  <meta name="product:brand" content="Atelier Demo"/>
  <meta name="product:color" content="sand"/>
  <meta name="product:material" content="linen"/>
</head>
<body>
  <h1>Utility Jacket</h1>
  <p>Perfect for breezy evenings.</p>
</body>
</html>
'''

        # Mocked fetch: pretend this is the response body from a retailer
        test_url = 'https://retailer.example.com/products/utility-jacket'
        raw_metadata = parse_product_html(fixture_html, test_url)
        print('Parsed raw metadata:', raw_metadata)

        # Map to a validated WardrobeItem and store it
        ingested_item = map_raw_metadata_to_wardrobe_item(USER_ID, test_url, raw_metadata)
        store.create_item(ingested_item)

        print('
Wardrobe after ingestion:')
        for item in store.list_items_for_user(USER_ID):
            print(f"- {item.item_id}: {item.category}/{item.sub_category} colors={item.colors} brand={item.brand}")


## Mood and basic outfit building (Phase Three)

Load mood profiles, exercise color theory helpers, and build a simple outfit using only wardrobe data (no calendar or weather context).


In [ ]:
        # Inspect available mood styles
        for mood in MOODS:
            profile = get_mood_style(mood)
            print(f"Mood '{profile.name}' -> styles {profile.style_tags}, palette {profile.palette}, background {profile.background_color}")

        # Color theory helpers on example palettes
        print('
Color theory checks:')
        print("Monochrome ['red', 'Red']:", monochrome(['red', 'Red']))
        print('Complementary blue vs orange:', complementary('blue', 'orange'))
        print('Analogous triplet [red, orange, yellow]:', analogous_triplet(['red', 'orange', 'yellow']))
        harmony = choose_harmonious_colors(['blue', 'white', 'black'], ['blue', 'gold'])
        print('Chosen harmony:', harmony)


In [ ]:
        # Build an outfit without calendar/weather context
        mood_profile = get_mood_style('casual')

        # Select candidates aligned with the mood
        candidates: CandidateSelectionResult = select_candidates_for_mood(USER_ID, mood_profile.name, wardrobe_tools)
        print('Candidate selection diagnostics:', candidates.diagnostics)

        # Build an outfit from the candidates
        outfit: OutfitBuildResult = build_outfit(candidates.items, mood_profile)
        print('
Chosen outfit item ids:', [item.item_id for item in outfit.items])
        print('Diagnostics:', outfit.diagnostics)

        # Apply harmony tweaks and generate a collage specification
        harmonised: HarmonyApplicationResult = apply_color_harmony(outfit.items, mood_profile)
        collage: CollageSpecResult = generate_collage_spec(harmonised.items, mood_profile)
        print('
Collage spec:')
        print(collage.collage)


## Calendar and weather context (Phase Four)

Use mock providers to create schedule and weather profiles, then synthesize a daily context. This keeps the workflow offline while showing how context is derived.


In [ ]:
        config = ADKConfig(project_id='manual-demo', api_key='dummy')

        # Calendar mock covering a single day range
        target_day = date.today() + timedelta(days=2)
        mock_events = [
            CalendarEvent(title='Morning gym', start_time=datetime.combine(target_day, datetime.min.time()).replace(hour=7), end_time=datetime.combine(target_day, datetime.min.time()).replace(hour=8)),
            CalendarEvent(title='Project meeting', start_time=datetime.combine(target_day, datetime.min.time()).replace(hour=10), end_time=datetime.combine(target_day, datetime.min.time()).replace(hour=11)),
            CalendarEvent(title='Dinner downtown', start_time=datetime.combine(target_day, datetime.min.time()).replace(hour=19), end_time=datetime.combine(target_day, datetime.min.time()).replace(hour=20)),
        ]
        calendar_agent = CalendarAgent(config=config, provider=MockCalendarProvider(mock_events))
        schedule_profile = calendar_agent.get_schedule_profile(user_id=USER_ID, target_date=target_day)
        print('Schedule profile (user-facing):', schedule_profile['user_facing_summary'])
        print('Debug summary:', schedule_profile['debug_summary'])

        # Weather mock
        mock_weather_profile = WeatherProfile(
            temp_min=6.0,
            temp_max=11.0,
            precipitation_probability=0.6,
            wind_speed=18.0,
            weather_condition='rain',
            clothing_guidance='Coat with hood',
        )
        weather_agent = WeatherAgent(config=config, provider=MockWeatherProvider(mock_weather_profile))
        weather_profile = weather_agent.get_weather_profile(user_id=USER_ID, location='Amsterdam', target_date=target_day)
        print('
Weather profile (labels):', {k: v for k, v in weather_profile.items() if k != 'raw_forecast'})

        # Combine into a daily context
        daily_context = synthesize_context(schedule_profile, weather_profile)
        print('
Daily context:', daily_context)


## Context aware outfit recommendation (Phase Five)

Filter wardrobe items using context signals, build outfits, score them, and generate collage specs. Each filtering step prints diagnostics so you can see how many items remain after each rule set.


In [ ]:
        # Start from the latest wardrobe items
        all_items = store.list_items_for_user(USER_ID)
        print(f'Initial item count: {len(all_items)}')

        # Weather-based filtering
        weather_filtered = filter_by_weather(all_items, weather_profile)
        print('Weather filter ->', weather_filtered.debug)

        # Formality filtering
        formality_filtered = filter_by_formality(weather_filtered.items, schedule_profile)
        print('Formality filter ->', formality_filtered.debug)

        # Movement filtering
        movement_filtered = filter_by_movement(formality_filtered.items, schedule_profile)
        print('Movement filter ->', movement_filtered.debug)

        # Mood filtering
        mood_filtered = filter_by_mood(movement_filtered.items, mood_profile)
        print('Mood filter ->', mood_filtered.debug)

        # Build multiple outfits from filtered pool
        filtered_candidates = list(mood_filtered.items)

        outfits: List[OutfitBuildResult] = []
        if len(filtered_candidates) >= 3:
            base_build = build_outfit(filtered_candidates, mood_profile)
            outfits.append(base_build)
            # Alternate mood to show variation
            urban_profile = get_mood_style('urban')
            outfits.append(build_outfit(filtered_candidates, urban_profile))

        print('
Built outfits count:', len(outfits))
        for idx, built in enumerate(outfits, start=1):
            score = score_outfit(built.items, mood_profile)
            print(f"Outfit {idx}: items={[item.item_id for item in built.items]}, score={score['total_score']}, breakdown={score}")
            collage_spec = generate_collage_spec(built.items, mood_profile)
            print('Collage background:', collage_spec.collage.get('background_color'))


## Full orchestrator end to end call

Instantiate the orchestrator with mock calendar and weather providers, then run a full request covering calendar context, weather context, context synthesis, and stylist recommendation. The output shows the ranked outfits, scores, collage specs, and debug summaries so teammates can trace the entire flow.


In [ ]:
        # Build stylist agent with the same local store
        stylist_agent = OutfitStylistAgent(config=config, wardrobe_tools=wardrobe_tools)

        # Wire mock providers into calendar and weather agents
        calendar_agent_full = CalendarAgent(config=config, provider=MockCalendarProvider(mock_events))
        weather_agent_full = WeatherAgent(config=config, provider=MockWeatherProvider(mock_weather_profile))

        # Orchestrator combines context and styling
        orchestrator = OrchestratorAgent(
            config=config,
            tools=wardrobe_tools.tool_defs(),
            stylist_agent=stylist_agent,
            calendar_agent=calendar_agent_full,
            weather_agent=weather_agent_full,
        )

        # Simulate a user request
        request_date = (date.today() + timedelta(days=3)).isoformat()
        full_response = orchestrator.plan_outfit(
            user_id=USER_ID,
            date=request_date,
            location='Amsterdam',
            mood='casual',
        )

        print('Request:', full_response.get('request'))
        print('
Schedule profile:', full_response.get('debug_summary', {}).get('schedule_profile'))
        print('Weather profile:', full_response.get('debug_summary', {}).get('weather_profile'))
        print('Daily context:', full_response.get('context'))
        print('
Top outfits:')
        for outfit in full_response.get('top_outfits', []):
            print({'items': [item['item_id'] for item in outfit.get('items', [])], 'score': outfit.get('score')})
            print('Collage:', outfit.get('collage'))

        print('
User-facing summary:', full_response.get('user_facing_summary'))
        print('Debug summary keys:', list(full_response.get('debug_summary', {}).keys()))
